# 供应网络设计 2

## 目标和前提条件

在本示例中，让我们将您的供应链网络设计技能提升到更高水平。我们将向您展示如何在给定一组工厂、仓库和客户的情况下，使用数学优化来确定应该开设或关闭哪些仓库以最小化总成本。

此模型是 H. Paul Williams 所著《数学规划中的模型构建》第五版第275-276页和332-333页中的示例20。

这是一个初级难度的示例；我们假设您了解 Python 并对 Gurobi Python API 和构建数学优化模型有一定了解。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

在这个问题中，我们有六个最终客户，每个客户对产品都有已知的需求。客户需求可以从六个仓库中的任何一个得到满足，或直接从两个工厂获得供应。每个仓库都有通过它的最大产品流量限制，每个工厂都有最大生产量限制。从工厂到仓库、从仓库到客户或从工厂直接到客户运输产品都有已知的成本。这个扩展版本提供了在六个可能的仓库中选择四个开设的机会。它还提供了在一个特定仓库扩大容量的选择。

我们的供应网络有两个工厂，分别位于利物浦和布莱顿，生产一种产品。每个工厂都有最大生产能力：

| 工厂 | 供应量（吨） |
| --- | --- |
| 利物浦 | 150,000 |
| 布莱顿 | 200,000 |

产品可以从工厂运送到六个仓库中的任何一个。每个仓库都有最大吞吐量。仓库不生产或消耗产品；它们只是将产品传递给客户。

| 仓库 | 吞吐量（吨） |
| --- | --- |
| 纽卡斯尔 | 70,000 |
| 伯明翰 | 50,000 |
| 伦敦 | 100,000 |
| 埃克塞特 | 40,000 |
| 布里斯托尔 | 30,000 |
| 北安普顿 | 25,000 |

实际上，我们只能在六个仓库中选择四个开设。开设仓库需要成本：

| 仓库 | 开设成本 |
| --- | --- |
| 纽卡斯尔 | 10,000 |
| 埃克塞特 | 5,000 |
| 布里斯托尔 | 12,000 |
| 北安普顿 | 4,000 |

（注意：书中提到开设布里斯托尔或北安普顿的成本，以及关闭纽卡斯尔或埃克塞特的节省，但这些只是表达同一选择的不同方式）。

我们还可以选择以3000美元的成本将伯明翰的容量扩大20,000吨。

我们的网络有六个客户，每个客户都有给定的需求。

| 客户 | 需求（吨） |
| --- | --- |
| C1 | 50,000 |
| C2 | 10,000 |
| C3 | 40,000 |
| C4 | 35,000 |
| C5 | 60,000 |
| C6 | 20,000 |

运输成本在下表中给出（单位：美元/吨）。列为源城市，行为目的地城市。例如，从利物浦运送产品到伦敦的成本为每吨1美元。表中的'-'表示该组合不可行，例如无法从布莱顿工厂运送到纽卡斯尔仓库。

| 目的地 | 利物浦 | 布莱顿 | 纽卡斯尔 | 伯明翰 | 伦敦 | 埃克塞特 | 布里斯托尔 | 北安普顿
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 仓库 |
| 纽卡斯尔 | 0.5 | - |
| 伯明翰 | 0.5 | 0.3 |
| 伦敦 | 1.0 | 0.5 |
| 埃克塞特 | 0.2 | 0.2 |
| 布里斯托尔 | 0.6 | 0.4 |
| 北安普顿 | 0.4 | 0.3 |
| 客户 |
| C1 | 1.0 | 2.0 | - | 1.0 | - | - | 1.2 | - |
| C2 | - | - | 1.5 | 0.5 | 1.5 | - | 0.6 | 0.4 |
| C3 | 1.5 | - | 0.5 | 0.5 | 2.0 | 0.2 | 0.5 | - |
| C4 | 2.0 | - | 1.5 | 1.0 | - | 1.5 | - | 0.5 |
| C5 | - | - | - | 0.5 | 0.5 | 0.5 | 0.3 | 0.6 |
| C6 | 1.0 | - | 1.0 | - | 1.5 | 1.5 | 0.8 | 0.9 |

需要回答的问题是：(i) 应该开设哪四个仓库？(ii) 是否应该扩大伯明翰的容量？(iii) 应该使用哪些仓库来满足客户需求？

---
## 模型构建

### 集合和索引

$f \in \text{工厂}=\{\text{利物浦}, \text{布莱顿}\}$

$d \in \text{仓库}=\{\text{纽卡斯尔}, \text{伯明翰}, \text{伦敦}, \text{埃克塞特}, \text{布里斯托尔}, \text{北安普顿}\}$

$c \in \text{客户}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{城市} = \text{工厂} \cup \text{仓库} \cup \text{客户}$

### 参数

$\text{成本}_{s,t} \in \mathbb{R}^+$: 从源点 $s$ 到目的地 $t$ 运输一吨产品的成本。

$\text{供应}_f \in \mathbb{R}^+$: 工厂 $f$ 的最大可能供应量（吨）。

$\text{吞吐量}_d \in \mathbb{R}^+$: 仓库 $d$ 的最大可能流通量（吨）。

$\text{需求}_c \in \mathbb{R}^+$: 客户 $c$ 的产品需求量（吨）。

$\text{开设成本}_d \in \mathbb{R}^+$: 开设仓库 $d$ 的成本（美元）。

### 决策变量

$\text{流量}_{s,t} \in \mathbb{N}^+$: 从源点 $s$ 到目的地 $t$ 运送的产品数量（吨）。

$\text{开设}_{d} \in [0,1]$: 仓库 $d$ 是否开设？

$\text{扩容} \in [0,1]$: 是否应该扩大伯明翰的容量？

### 目标函数

- **成本**: 最小化总运输成本加上开设仓库的成本。

\begin{equation}
\text{最小化} \quad Z = \sum_{(s,t) \in \text{城市} \times \text{城市}}{\text{成本}_{s,t}*\text{流量}_{s,t}} +
                          \sum_{{d} \in \text{仓库}}{\text{开设成本}_d*\text{开设}_d} +
                          3000 * \text{扩容}
\end{equation}

### 约束条件

- **工厂产出**: 从工厂流出的产品量必须遵守最大产能限制。

\begin{equation}
\sum_{t \in \text{城市}}{\text{流量}_{f,t}} \leq \text{供应}_{f} \quad \forall f \in \text{工厂}
\end{equation}

- **客户需求**: 产品流量必须满足客户需求。

\begin{equation}
\sum_{s \in \text{城市}}{\text{流量}_{s,c}} = \text{需求}_{c} \quad \forall c \in \text{客户}
\end{equation}

- **仓库流量**: 进入仓库的流量等于离开仓库的流量。

\begin{equation}
\sum_{s \in \text{城市}}{\text{流量}_{s,d}} = 
\sum_{t \in \text{城市}}{\text{流量}_{d,t}}
\quad \forall d \in \text{仓库}
\end{equation}

- **仓库容量（除伯明翰外）**: 进入仓库的流量必须遵守仓库容量限制，且仅当仓库开设时才允许有流量。

\begin{equation}
\sum_{s \in \text{城市}}{\text{流量}_{s,d}} \leq \text{吞吐量}_{d} * \text{开设}_{d}
\quad \forall d \in \text{仓库} - \text{伯明翰}
\end{equation}

- **仓库容量（伯明翰）**: 进入伯明翰的流量必须遵守仓库容量限制，该容量可能已被扩大。

\begin{equation}
\sum_{s \in \text{城市}} \text{流量}_{s,\text{伯明翰}} \leq \text{吞吐量}_{\text{伯明翰}} + 20000 * \text{扩容}
\end{equation}

- **开设仓库**: 最多开设4个仓库（伯明翰和伦敦没有选择）。

\begin{equation}
\sum_{d \in \text{仓库}}{\text{开设}_{d}} \leq 4
\end{equation}

\begin{equation}
\text{开设}_{\text{伯明翰}} = \text{开设}_{\text{伦敦}} = 1
\end{equation}

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# 测试环境：Python 3.11 & Gurobi 11.0

## 输入数据
我们定义模型的所有输入数据。

In [2]:
# 创建字典来存储工厂供应限制、仓库吞吐量限制、开设仓库成本和客户需求

supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000,
                'Bristol': 30000,
                'Northampton': 25000})

opencost = dict({'Newcastle': 10000,
                 'Birmingham': 0,
                 'London': 0,
                 'Exeter': 5000,
                 'Bristol': 12000,
                 'Northampton': 4000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# 创建字典来存储运输成本

arcs, cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'Bristol'): 0.6,
    ('Liverpool', 'Northampton'): 0.4,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'Bristol'): 0.4,
    ('Brighton', 'Northampton'): 0.3,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5,
    ('Bristol', 'C1'): 1.2,
    ('Bristol', 'C2'): 0.6,
    ('Bristol', 'C3'): 0.5,
    ('Bristol', 'C5'): 0.3,
    ('Bristol', 'C6'): 0.8,
    ('Northampton', 'C2'): 0.4,
    ('Northampton', 'C4'): 0.5,
    ('Northampton', 'C5'): 0.6,
    ('Northampton', 'C6'): 0.9
})

## 模型部署

我们创建模型和变量。'flow'变量简单地捕获在源点和目的地之间允许路径上流动的产品数量。'open'变量捕获关于开设哪些仓库的决策。'expand'变量捕获是否扩大伯明翰容量的选择。这里提供了目标系数，所以我们后面不需要再提供优化目标。

In [3]:
model = gp.Model('SupplyNetworkDesign2')

depots = through.keys()
flow = model.addVars(arcs, obj=cost, name="flow")
open = model.addVars(depots, obj=opencost, vtype=GRB.BINARY, name="open")
expand = model.addVar(obj=3000, vtype=GRB.BINARY, name="expand")

open['Birmingham'].lb = 1
open['London'].lb = 1
model.objcon = -(opencost['Newcastle'] + opencost['Exeter']) # 表达为“关闭的节省”

Set parameter LicenseID to value 2601452


我们的第一个约束要求离开工厂的总流量不能超过该工厂的供应能力。

In [4]:
# 生产能力限制

factories = supply.keys()
factory_flow = model.addConstrs((gp.quicksum(flow.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory")

我们的下一个约束要求进入客户的总流量必须等于该客户的需求量。

In [5]:
# 客户需求

customers = demand.keys()
customer_flow = model.addConstrs((gp.quicksum(flow.select('*', customer)) == demand[customer]
                                  for customer in customers), name="customer")

我们的最后一组约束与仓库有关。第一个约束要求进入仓库的总产品量必须等于离开的总量。

In [6]:
# 仓库流量守恒

depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

第二组约束限制通过仓库的产品量最多等于该仓库的吞吐量，如果仓库未开设则为0。

In [7]:
# 仓库吞吐量

all_but_birmingham = list(set(depots) - set(['Birmingham']))

depot_capacity = model.addConstrs((gp.quicksum(flow.select(depot, '*')) <= through[depot]*open[depot]
                                   for depot in all_but_birmingham), name="depot_capacity")


伯明翰的容量约束不同。该仓库始终开放，但我们可以选择扩大其容量。

In [8]:
birmingham_capacity = model.addConstr(gp.quicksum(flow.select('*', 'Birmingham')) <= through['Birmingham'] +
                                      20000*expand, name="birmingham_capacity")

最后，最多只能开设4个仓库

In [9]:
# 仓库数量限制

depot_count = model.addConstr(open.sum() <= 4)

现在我们优化模型

In [10]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 21 rows, 49 columns and 119 nonzeros
Model fingerprint: 0xab94e1c9
Variable types: 42 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+05]
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolved: 20 rows, 47 columns, 113 nonzeros
Variable types: 42 continuous, 5 integer (5 binary)
Found heuristic solution: objective 174000.00000

Root relaxation: cutoff, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0    

---
## 分析

通过在北安普顿开设仓库，关闭纽卡斯尔的仓库，并扩大伯明翰的仓库容量，我们所有客户的产品需求都可以以总成本174,000美元得到满足：

In [11]:
print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')

List of open depots: ['Birmingham', 'London', 'Exeter', 'Northampton']
Expand Birmingham


In [12]:
product_flow = pd.DataFrame(
    [{"From": arc[0], "To": arc[1], "Flow": flow[arc].x} for arc in arcs if flow[arc].x > 1e-6]
)
product_flow.index=[''] * len(product_flow)
product_flow

,From,To,Flow
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,70000.0
,Brighton,London,10000.0
,Brighton,Exeter,40000.0
,Brighton,Northampton,25000.0
,Birmingham,C2,10000.0
,Birmingham,C4,10000.0
,Birmingham,C5,50000.0
,London,C5,10000.0


---
## 参考文献

H. Paul Williams 著《数学规划中的模型构建》第五版。

Copyright © 2020 Gurobi Optimization, LLC